### Simulating, visualizing, and quantifying the information loss in scarps associated with landscape evolution

Author: Alba M. Rodriguez Padilla

### load Python packages

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from pylab import show, figure, plot
import time
from landlab import RasterModelGrid
from landlab.plot.imshow import imshow_grid
from landlab.io import read_esri_ascii
import matplotlib.pyplot as plt
import os
import geopandas as gpd
from scipy import stats
from osgeo import gdal
import glob
from PIL import Image
import matplotlib.cbook as cbook
import scipy.optimize
from scipy.optimize import curve_fit
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib as mpl
mpl.rcParams['font.size'] = 8
import utils
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
# check if Figures/ and Code output csv/ directories exist, otherwise create them
if not os.path.exists('Figures'):
    os.makedirs('Figures')
    
if not os.path.exists('Code output csv'):
    os.makedirs('Code output csv')

### User-defined features

In [3]:
n_iter = np.array(
    [0, 2, 20, 100, 200]
)  # number of iterations (total_time = n_iter * dt) at which to produce output,

# input file locations
DEM_dir = "DEMs/"
fault_trace_shapefile_dir = "Shps_fault_traces/"
FZW_shapefile_dir = "Shps_FZW/"
epsg_code = 32611  # Ridgecrest and EMC UTM 11N
excel_filename = "Code output csv/information_loss_analysis_outputs.csv"
linear_or_nonlinear = (
    "linear"  # choice to use linear or nonlinear (Taylor) diffuser in landlab
)

save_fig_YN = "Yes"  # if Yes, all pdfs get saved to Figures/ directory
save_data_excel_YN = "Yes"  # if Yes, info gets appended to csv file

if save_data_excel_YN == "Yes":  # clean up older file
    if os.path.exists(excel_filename):
        os.remove(excel_filename)
        
initial_slopes = [] # save in text file for soil_flux_slope_comparison background histogram 

### Plot landscape evolution, line evolution, height evolution, and slope evolution over time

In [4]:
DEMs_shapefiles = utils.load_shapefiles_for_DEMs(DEM_dir, fault_trace_shapefile_dir, FZW_shapefile_dir)

for dem_name, shapefile_data in DEMs_shapefiles.items():
    shapefiles_fault_traces = shapefile_data['fault_traces']
    shapefiles_FZW = shapefile_data['width_lines']
    
    # Extract DEM ID
    first_char = dem_name[0]
    numeric_chars = "".join(filter(str.isdigit, dem_name))
    DEMID = first_char + numeric_chars

    if linear_or_nonlinear == "linear":
        line_length, line_width, coeff_t, years_t, initial_slopes = (
            utils.plot_evolution_time_linear(
                n_iter,
                dem_name,
                shapefiles_fault_traces,
                shapefiles_FZW,
                epsg_code,
                save_fig_YN,
                initial_slopes,
                D=0.001,
            )
        )
        diffusion_type = "linear"
    elif linear_or_nonlinear == "nonlinear":
        line_length, coeff_t, years_t, _ = utils.plot_evolution_time_nonlinear(
            n_iter, dem_name, shapefiles_fault_traces, shapefiles_FZW, epsg_code, save_fig_YN, [], D=0.001
        )
        diffusion_type = "nonlinear"
    else:
        NameError("User must choose linear or nonlinear diffusion")
    plt.close() # so that I can upload file to github without huge memory sink
    print(dem_name)
    first_char = dem_name[0]
    numeric_chars = "".join(filter(str.isdigit, dem_name))
    DEMID = first_char + numeric_chars

    if first_char == "R":
        location = "Ridgecrest"
    elif first_char == "E":
        location = "EMC"
    else:
        NameError("File name must start with R or E")

    repeated_locations = [location for _ in line_length]
    dem_ID = [DEMID for _ in line_length]

    combined_data = {
        "Length (m)": line_length,
        "Width (m)": line_width,
        "Degradation coefficient": coeff_t,
        "Diffusion type": diffusion_type,
        "Time (years)": years_t,
        "DEM ID": dem_ID,
        "Location": repeated_locations,
    }

    df = pd.DataFrame(combined_data)

    if save_data_excel_YN == "Yes":
        dem_row = df

        if not os.path.exists(excel_filename):
            data_to_save = df
        else:
            data_to_save = pd.read_csv(excel_filename)
            data_to_save = pd.concat([data_to_save, dem_row], ignore_index=True)

        data_to_save.to_csv(excel_filename, index=False)
        if linear_or_nonlinear == "linear" and initial_slopes:
            initial_slopes_array = np.concatenate(initial_slopes)
            np.savetxt('Code output csv/initial_slopes.txt', initial_slopes_array)
        

E9
E8
R1
R3
R2
R6
R7
R5
R4
R10
E10
R9
E1
R8
E3
E2
E6
E7
E5
E4
